In [1]:
import os
import gc
import re
import sys
import math
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np
from xpinyin import Pinyin

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [95]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# reload(sys)
# sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
# PRODUCE_DB_USER = 'pingjia'
# PRODUCE_DB_PASSWD = 'De32wsxC'

PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'

PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://' + PRODUCE_DB_USER + ':' + PRODUCE_DB_PASSWD + '@' + PRODUCE_DB_ADDR_OUTTER + '/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE)

# query_sql = 'select * from pingjia.open_model_detail '
query_sql = 'select * from pingjia.open_category '
# query_sql = 'select source_id,detail_name,year,month,mile,price,city,domain,transfer_owner from crawler_car_source'
origin_train = pd.read_sql_query(query_sql, engine)
origin_train.to_csv('./origin_open_category.csv', index=False)
print('finish query, num:'+str(len(origin_train)))

finish query, num:2033


### 一.车系匹配

In [49]:
def process_brand_name(df):
    """
    删除没用的字符
    """
    text = df['brand_name']
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('MINI迷你', 'MINI')
    text = text.replace('RUF如虎', '如虎')
    text = text.replace('Pagani', '帕加尼')
    text = text.replace('柯尼赛格', '科尼赛克')
    text = text.replace('布嘉迪', '布加迪')
    text = text.replace('幻速', '北汽幻速')
    text = text.replace('昌河', '北汽昌河')
    text = text.replace('汽车', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_model_name(df):
    """
    删除没用的字符
    """
    text = df['model_name']
    text = text.lower()
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('-', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('/', '')
    text = text.replace('°', '')
    text = text.replace('!', '')
    text = text.replace('ⅰ', 'i')
    text = text.replace('ⅱ', 'ii')
    text = text.replace('ⅲ', 'iii')
    text = text.replace('pagani', '帕加尼')
    text = text.replace('柯尼赛格', '科尼赛克')
    text = text.replace('科尼赛格', '科尼赛克')
    text = text.replace('布嘉迪', '布加迪')
    text = text.replace('吉利康迪全球鹰', '全球鹰')
    text = text.replace('汽车', '')
    if df['brand_name'] == '长丰猎豹':
        text = text.replace('猎豹', '长丰猎豹')
    if df['brand_name'] == '北汽制造':
        text = text.replace('北汽', '')
    if df['brand_name'] == '北汽绅宝':
        text = text.replace('北京绅宝', '北汽绅宝')
    if df['brand_name'] == '北汽幻速':
        text = text.replace('幻速', '北汽幻速')
    if df['brand_name'] == '北汽昌河':
        text = text.replace('昌河', '北汽昌河')
    text = text.replace(' ', '')
    # text = text.lower()
    return text

def process_final_text(df):
    """
    删除没用的字符
    """
    text = df['model_name'].replace(df['brand_name'], '')
    # text = text.lower()
    return df['brand_name']+text

In [51]:
open_category = pd.read_csv('../tmp/train/open_category.csv')
open_category = open_category.loc[~(open_category['status'] == 'D'), :].reset_index(drop=True)

# 人工解决的车系问题
open_category.loc[(open_category['id'] == 204), 'name'] = '奔驰A级(进口)'
open_category = open_category.loc[~(open_category['id'].isin([1069,1264])), :] 

liyang_brand = open_category.loc[(open_category['parent'].isnull()), ['id', 'name', 'slug']].rename(columns={'name': 'brand_name', 'slug': 'parent'}).reset_index(drop=True)
liyang_model = open_category.loc[(open_category['parent'].notnull()), ['id', 'name', 'slug', 'parent']].rename(columns={'name': 'model_name',
             'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
liyang_model = liyang_model.merge(liyang_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
liyang_model = liyang_model.loc[:, ['brand_id', 'model_id', 'brand_name', 'model_name','parent', 'model_slug']]

liyang_model['brand_name'] = liyang_model.apply(process_brand_name, axis=1)
liyang_model['model_name'] = liyang_model.apply(process_model_name, axis=1)
liyang_model['final_text'] = liyang_model.apply(process_final_text, axis=1)
liyang_model = liyang_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
liyang_model = liyang_model.rename(columns={'brand_id':'liyang_brand_id','model_id':'liyang_model_id','parent':'brand_slug',
                                                       'brand_name':'liyang_brand_name','model_name':'liyang_model_name'})
print('力洋车系数量:',len(liyang_model))

verify = liyang_model.groupby(['final_text'])['model_slug'].count().reset_index().rename(columns={'model_slug':'num'})
print('车系名称重复的车系,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])
liyang_model.to_csv('./man_liyang.csv', index=False)

力洋车系数量: 1763
车系名称重复的车系,请人工检查:
Empty DataFrame
Columns: [final_text, num]
Index: []


In [2]:
def process_brand_name(df):
    """
    删除没用的字符
    """
    text = df['brand_name']

    text = text.replace('Jeep', '吉普')
    text = text.replace('SWM斯威汽车', '斯威')
    text = text.replace('名爵', 'MG')
    text = text.replace('上汽大通', '大通')
    text = text.replace('福汽启腾', '启腾')
    text = text.replace('汽车', '')
    text = text.replace('・', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_model_name(df):
    """
    删除没用的字符
    """
    text = df['model_name']
    text = text.lower()
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('-', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('/', '')
    text = text.replace('°', '')
    text = text.replace('!', '')
    text = text.replace('名爵', 'mg')
    text = text.replace('上汽大通', '大通')
    text = text.replace('风行', '东风风行')
    text = text.replace('swm斯威', '斯威')
    if df['brand_name'] == '北汽制造':
        text = text.replace('北汽', '')
    if df['brand_name'] == '北汽绅宝':
        text = text.replace('绅宝', '北汽绅宝')
    text = text.replace('汽车', '')
    text = text.replace(' ', '')

    return text

def process_final_text(df):
    """
    删除没用的字符
    """
    text = df['model_name'].replace(df['brand_name'], '')
    # text = text.lower()
    return df['brand_name']+text

In [3]:
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_model = car_autohome_all.loc[:, ['brand_slug', 'model_slug', 'brand_name', 'model_name']]
car_autohome_model = car_autohome_model.drop_duplicates(['brand_slug', 'model_slug']).reset_index(drop=True)
car_autohome_model['brand_name'] = car_autohome_model.apply(process_brand_name, axis=1)
car_autohome_model['model_name'] = car_autohome_model.apply(process_model_name, axis=1)
car_autohome_model['final_text'] = car_autohome_model.apply(process_final_text, axis=1)
car_autohome_model = car_autohome_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
car_autohome_model = car_autohome_model.rename(columns={'brand_slug':'car_autohome_brand_id','model_slug':'car_autohome_model_id',
                                                       'brand_name':'car_autohome_brand_name','model_name':'car_autohome_model_name'})
print('汽车之家车系数量:',len(car_autohome_model))

verify = car_autohome_model.groupby(['final_text'])['car_autohome_model_id'].count().reset_index().rename(columns={'car_autohome_model_id':'num'})
print('车系名称重复的车系,请人工检查:')
print(verify.loc[(verify['num'] > 1),:])

car_autohome_model.to_csv('./man_car_autohome.csv', index=False)

汽车之家车系数量: 1882
车系名称重复的车系,请人工检查:
Empty DataFrame
Columns: [final_text, num]
Index: []


#### 车系匹配

In [44]:
def get_slug(df,origin_combine_detail,origin_combine_model):
    temp = origin_combine_detail.loc[(origin_combine_detail['car_autohome_detail_id'] == df['detail_slug']),:].reset_index(drop=True)
    
    origin_brand_slug = temp['brand_slug'][0]
    origin_model_slug = temp['model_slug'][0]
    origin_model_detail_slug = temp['model_detail_slug'][0]
    
    if df['model_slug'] != temp['car_autohome_model_id'][0]:
        origin_model_slug = origin_combine_model.loc[(origin_combine_model['car_autohome_model_id'] == df['model_slug']),['model_slug']].values
        if len(origin_model_slug) == 0:
            origin_model_slug = 'car_autohome_model_'+str(df['model_slug'])
        else:
            origin_model_slug = origin_model_slug[0][0]
            
    if df['brand_slug'] != temp['car_autohome_brand_id'][0]:
        origin_brand_slug = origin_combine_model.loc[(origin_combine_model['car_autohome_brand_id'] == df['brand_slug']),['brand_slug']].values
        if len(origin_brand_slug) == 0:
            origin_brand_slug = 'car_autohome_brand_'+str(df['brand_slug'])
        else:
            origin_brand_slug = origin_brand_slug[0][0]
            
    return pd.Series([origin_brand_slug,origin_model_slug,origin_model_detail_slug])
    
# origin_combine_model = pd.read_csv('./origin_open_category.csv')
# origin_combine_model = origin_combine_model.loc[(origin_combine_model['status'] != 'D'),:].reset_index(drop=True)
# origin_brand = origin_combine_model.loc[(origin_combine_model['parent'].isnull()), ['id', 'slug']].rename(columns={'slug': 'parent'}).reset_index(drop=True)
# origin_model = origin_combine_model.loc[(origin_combine_model['parent'].notnull()), ['id', 'slug', 'parent']].rename(columns={'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
# origin_model = origin_model.merge(origin_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
# origin_model = origin_model.loc[:, ['brand_id', 'model_id', 'parent', 'model_slug']]
# origin_model = origin_model.rename(columns={'brand_id':'liyang_brand_id','model_id':'liyang_model_id','parent':'brand_slug',})

# origin_open_model_detail = pd.read_csv('./origin_open_model_detail.csv').rename(columns={'global_slug':'model_slug','detail_model_slug':'model_detail_slug'})
# origin_open_model_detail = origin_open_model_detail.loc[(origin_open_model_detail['status'] != 'D'),:].reset_index(drop=True)
# origin_open_model_detail = origin_open_model_detail.loc[:,['id','model_slug','model_detail_slug']]
# origin_open_model_detail = origin_open_model_detail.merge(origin_model,how='left',on=['model_slug'])

combine_brand = pd.read_csv('./new_combine_brand.csv', low_memory=False)
combine_brand = combine_brand.loc[:, ['slug','car_autohome_brand_id']].rename(columns={'slug':'brand_slug'})
combine_model = pd.read_csv('./new_combine_model.csv', low_memory=False)
combine_model = combine_model.loc[:, ['slug','parent','car_autohome_model_id']].rename(columns={'slug':'model_slug','parent':'brand_slug'})
combine_detail = pd.read_csv('./new_combine_detail.csv', low_memory=False)
combine_detail = combine_detail.loc[:, ['detail_model_slug','global_slug','car_autohome_detail_id']].rename(columns={'detail_model_slug':'model_detail_slug','global_slug':'model_slug'})
combine_model = combine_model.merge(combine_brand,how='left',on=['brand_slug'])
combine_detail = combine_detail.merge(combine_model,how='left',on=['model_slug'])

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all[['origin_brand_slug','origin_model_slug','origin_model_detail_slug']] = car_autohome_all.apply(get_slug, args=(combine_detail,combine_model,), axis=1)

In [63]:
def cal_num(df,all_detail):
    return pd.Series([len(all_detail.loc[(all_detail['model_slug'] == df['model_slug']),:]),len(all_detail.loc[(all_detail['origin_model_slug'] == df['origin_model_slug']),:])])

final = car_autohome_all.copy()
final = final.sort_values(by=['brand_slug','model_slug']).reset_index(drop=True)
model = final.loc[:, ['brand_slug','model_slug','origin_brand_slug','origin_model_slug']].reset_index(drop=True)
model = model.drop_duplicates(['model_slug','origin_model_slug']).reset_index(drop=True)

part1 = model.groupby(['brand_slug'])['model_slug'].count().reset_index()
part2 = model.groupby(['brand_slug'])['origin_model_slug'].count().reset_index()
part1['origin_model_slug'] = part2['origin_model_slug']
model[['model_slug_num','origin_model_slug_num']] = model.apply(cal_num, args=(final,), axis=1)
print('品牌车系数量不一致的情况:',len(part1.loc[(part1['model_slug']!=part1['origin_model_slug']),:]))
print('车系款型数量不一致的情况:',len(model.loc[(model['model_slug_num']!=model['origin_model_slug_num']),:]))

品牌车系数量不一致的情况: 0
车系款型数量不一致的情况: 0


In [9]:
origin_combine_model = pd.read_csv('./origin_open_category.csv')
origin_brand = origin_combine_model.loc[(origin_combine_model['parent'].isnull()), ['id', 'slug']].rename(columns={'slug': 'parent'}).reset_index(drop=True)
origin_model = origin_combine_model.loc[(origin_combine_model['parent'].notnull()), ['id', 'slug', 'parent']].rename(columns={'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
origin_model = origin_model.merge(origin_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
origin_model = origin_model.loc[:, ['brand_id', 'model_id', 'parent', 'model_slug']]
origin_model = origin_model.rename(columns={'brand_id':'liyang_brand_id','model_id':'liyang_model_id','parent':'brand_slug',})

origin_open_model_detail = pd.read_csv('./origin_open_model_detail.csv').rename(columns={'global_slug':'model_slug','detail_model_slug':'model_detail_slug'})
origin_open_model_detail = origin_open_model_detail.merge(origin_combine_model.loc[:,['id','model_slug','model_detail_slug']])

# combine_brand = pd.read_csv('../tmp/train/combine_brand.csv', low_memory=False)
# combine_model = pd.read_csv('../tmp/train/combine_model.csv', low_memory=False)
# combine_detail = pd.read_csv('../tmp/train/combine_detail.csv', low_memory=False)

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')


# origin_combine_model = origin_combine_model.merge(combine_model.loc[:,['slug','car_autohome_model_id']],how='left',on=['slug'])

# liyang_brand = origin_combine_model.loc[(origin_combine_model['parent'].isnull()), ['id', 'name', 'slug']].rename(columns={'name': 'brand_name', 'slug': 'parent'}).reset_index(drop=True)
# liyang_model = origin_combine_model.loc[(origin_combine_model['parent'].notnull()), ['id', 'name', 'slug', 'parent','car_autohome_model_id']].rename(columns={'name': 'model_name',
#              'slug': 'model_slug', 'id': 'model_id'}).reset_index(drop=True)
# liyang_model = liyang_model.merge(liyang_brand, how='left', on=['parent']).rename(columns={'id': 'brand_id'})
# liyang_model = liyang_model.loc[:, ['brand_id', 'model_id', 'brand_name', 'model_name','parent', 'model_slug','car_autohome_model_id']]
# liyang_model = liyang_model.sort_values(by=['brand_name', 'model_name']).reset_index(drop=True)
# liyang_model = liyang_model.rename(columns={'brand_id':'liyang_brand_id','model_id':'liyang_model_id','parent':'brand_slug',
#                                                        'brand_name':'liyang_brand_name','model_name':'liyang_model_name'})

# match_result = car_autohome_model.copy()
# match_result = match_result.merge(liyang_model.loc[:, ['liyang_brand_id','liyang_model_id','brand_slug','model_slug','car_autohome_model_id']], how='left', on=['car_autohome_model_id'])

# step1 = match_result.loc[(match_result['liyang_model_id'].notnull()),:].reset_index(drop=True)
# step2 = match_result.loc[(match_result['liyang_model_id'].isnull()),:].reset_index(drop=True)
# print('总数量:',len(match_result),'匹配数量:',len(step1),'未匹配数量:',len(step2))

In [123]:
def process_brand_slug(df):
    if type(df['brand_slug']) == float:
        temp = temp_match.loc[(temp_match['car_autohome_brand_name'] == df['car_autohome_brand_name'])&(temp_match['liyang_brand_id'].notnull()),:].reset_index(drop=True)
        if len(temp) > 0:
            liyang_brand_id = temp.loc[0,'liyang_brand_id']
            brand_slug = temp.loc[0,'brand_slug']
        else:
            liyang_brand_id = np.NaN
            brand_slug = 'car_autohome_brand_'+str(df['car_autohome_brand_id'])
    else:
        liyang_brand_id = df['liyang_brand_id']
        brand_slug = df['brand_slug']
    if type(df['model_slug']) == float:
        model_slug = 'car_autohome_model_'+str(df['car_autohome_model_id'])
    else:
        model_slug = df['model_slug']
    return pd.Series([liyang_brand_id,brand_slug,model_slug])


brand_count = match_result.loc[:,['car_autohome_brand_id','brand_slug']].copy()
brand_count = brand_count.drop_duplicates(['car_autohome_brand_id','brand_slug'])
brand_count = brand_count.groupby(['brand_slug'])['car_autohome_brand_id'].count().reset_index()
brand_count = brand_count.loc[(brand_count['car_autohome_brand_id'] > 1),:]
match_result.loc[(match_result['brand_slug'].isin(list(set(brand_count.brand_slug.values)))),['liyang_brand_id','brand_slug']] = np.NaN,np.NaN
temp_match = match_result.copy()
match_result[['liyang_brand_id','brand_slug','model_slug']] = match_result.apply(process_brand_slug,axis=1)
match_result.to_csv('./man_liyang_model_match_result.csv',index=False)
print(match_result.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880 entries, 0 to 1879
Data columns (total 9 columns):
car_autohome_brand_id      1880 non-null int64
car_autohome_model_id      1880 non-null int64
car_autohome_brand_name    1880 non-null object
car_autohome_model_name    1880 non-null object
final_text                 1880 non-null object
liyang_brand_id            1531 non-null float64
liyang_model_id            1851 non-null float64
brand_slug                 1880 non-null object
model_slug                 1880 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 146.9+ KB
None


In [129]:
def process_detail_slug(df):
    if type(df['model_detail_slug']) == float:
        return 'car_autohome_detail_'+str(df['detail_slug'])
    return df['model_detail_slug']

car_autohome_final = pd.read_csv('./man.csv')
car_autohome_final.loc[(car_autohome_final['cos_similar'] < 0.82),['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = np.NaN,np.NaN,np.NaN,np.NaN
car_autohome_final.loc[(abs(car_autohome_final['price_bn'] - car_autohome_final['liyang_price_bn']) > 1),['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = np.NaN,np.NaN,np.NaN,np.NaN
dup_detail = car_autohome_final.groupby(['model_detail_slug'])['liyang_detail_model'].count().reset_index()
dup_detail = dup_detail.loc[(dup_detail['liyang_detail_model'] > 1),'model_detail_slug'].values
car_autohome_final.loc[(car_autohome_final['model_detail_slug'].isin(dup_detail)),['liyang_detail_model','model_detail_slug','liyang_price_bn','cos_similar']] = np.NaN,np.NaN,np.NaN,np.NaN
car_autohome_final['model_detail_slug'] = car_autohome_final.apply(process_detail_slug,axis=1)
car_autohome_final = car_autohome_final.drop(['final_text','fill_vector','liyang_detail_model','liyang_price_bn','cos_similar'],axis=1)
car_autohome_final.to_csv('./man1.csv',index=False)
car_autohome_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32672 entries, 0 to 32671
Data columns (total 21 columns):
detail_slug          32672 non-null int64
brand_name           32672 non-null object
brand_slug           32672 non-null int64
model_name           32672 non-null object
model_slug           32672 non-null int64
detail_name          32672 non-null object
manufacturer         32672 non-null object
price_bn             32672 non-null float64
body                 32672 non-null object
energy               32672 non-null object
volume_extend        32672 non-null object
emission_standard    32672 non-null object
listed_year          32672 non-null int64
online_year          32672 non-null int64
volume               32672 non-null float64
control              32672 non-null object
liyang_brand_id      25869 non-null float64
liyang_model_id      32490 non-null float64
liyang_brand_slug    32672 non-null object
liyang_model_slug    32672 non-null object
model_detail_slug    32672 non-n

In [65]:
car_autohome_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32672 entries, 0 to 32671
Data columns (total 19 columns):
detail_slug                 32672 non-null int64
brand_name                  32672 non-null object
brand_slug                  32672 non-null int64
model_name                  32672 non-null object
model_slug                  32672 non-null int64
detail_name                 32672 non-null object
manufacturer                32672 non-null object
price_bn                    32672 non-null float64
body                        32672 non-null object
energy                      32672 non-null object
volume_extend               32672 non-null object
emission_standard           32672 non-null object
listed_year                 32672 non-null int64
online_year                 32672 non-null int64
volume                      32672 non-null float64
control                     32672 non-null object
origin_brand_slug           32672 non-null object
origin_model_slug           32672 non-null o

### 三.品牌车系款型组装

#### 1.品牌

In [85]:
def process_keywords(df):
    p = Pinyin()
    keywords = p.get_pinyin(df['name'], '')
    final_keywords = ','+keywords+','+df['name']+','+keywords
    first_letter = p.get_initial(df['name'][0])
    return pd.Series([final_keywords,first_letter,keywords])

origin_combine_brand = pd.read_csv('./new_combine_brand.csv')
origin_combine_brand = origin_combine_brand.loc[(origin_combine_brand['parent'].isnull()),:].reset_index(drop=True)
origin_combine_brand = origin_combine_brand.drop(['name','slug'],axis=1)
car_autohome_brand = car_autohome_all.loc[:,['origin_brand_slug','brand_name','brand_slug']].rename(columns={'origin_brand_slug':'slug','brand_slug':'car_autohome_brand_id'})
car_autohome_brand = car_autohome_brand.drop_duplicates(['car_autohome_brand_id'])
car_autohome_brand = car_autohome_brand.merge(origin_combine_brand,how='left',on=['car_autohome_brand_id']).rename(columns={'brand_name':'name'})
car_autohome_brand[['keywords', 'first_letter', 'pinyin']] = car_autohome_brand.apply(process_keywords, axis=1)
car_autohome_brand['brand_area'] = car_autohome_brand['brand_area'].fillna('国产')
car_autohome_brand['has_detailmodel'] = 1
car_autohome_brand['units'] = 0
car_autohome_brand['score'] = 0
car_autohome_brand['status'] = 'Y'
car_autohome_brand['popular'] = 'B'
car_autohome_brand['on_sale'] = '1'
car_autohome_brand.to_csv('./combine_brand.csv',index=False)

temp = car_autohome_brand.loc[:,['car_autohome_brand_id']]
temp = temp.drop_duplicates(['car_autohome_brand_id'])
print('最终去重之后品牌数:',len(temp))

temp = pd.read_csv('../tmp/train/car_autohome_all.csv',low_memory=False)
temp = temp.loc[:,['brand_slug']]
temp = temp.drop_duplicates(['brand_slug'])
print('汽车之家品牌数:',len(temp))
car_autohome_brand.info()

最终去重之后品牌数: 201
汽车之家品牌数: 201
<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 200
Data columns (total 29 columns):
slug                     201 non-null object
name                     201 non-null object
car_autohome_brand_id    201 non-null int64
id                       171 non-null float64
source_id                102 non-null float64
alias                    0 non-null float64
url                      98 non-null object
parent                   0 non-null float64
checker_runtime_id       86 non-null float64
keywords                 201 non-null object
classified               0 non-null float64
classified_url           0 non-null float64
slug_global              0 non-null float64
logo_img                 138 non-null object
mum                      0 non-null float64
first_letter             201 non-null object
has_detailmodel          201 non-null int64
starting_price           0 non-null float64
classified_slug          0 non-null float64
thumbnail            

#### 2.车系

In [88]:
origin_combine_model = pd.read_csv('./new_combine_model.csv')
# origin_combine_model = origin_combine_model.loc[(origin_combine_model['parent'].notnull()),:].reset_index(drop=True)
origin_combine_model = origin_combine_model.drop(['name','parent','classified','mum','slug'],axis=1)
car_autohome_model = car_autohome_all.loc[:,['origin_model_slug','model_name','origin_brand_slug','body', 'manufacturer','model_slug']].rename(columns={'origin_model_slug':'slug','origin_brand_slug':'parent','body':'classified','manufacturer':'mum','model_slug':'car_autohome_model_id'})
car_autohome_model = car_autohome_model.drop_duplicates(['car_autohome_model_id'])
car_autohome_model = car_autohome_model.merge(origin_combine_model,how='left',on=['car_autohome_model_id']).rename(columns={'model_name':'name'})
car_autohome_model[['keywords', 'first_letter', 'pinyin']] = car_autohome_model.apply(process_keywords, axis=1)
car_autohome_model.loc[(car_autohome_model['attribute'].isnull()), 'attribute'] = '国产'
car_autohome_model['normalized_name'] = car_autohome_model['name']
car_autohome_model['has_detailmodel'] = 1
car_autohome_model['units'] = 0
car_autohome_model['score'] = 0
car_autohome_model['status'] = 'Y'
car_autohome_model['popular'] = 'B'
car_autohome_model['on_sale'] = '1'
car_autohome_model.to_csv('./combine_model.csv',index=False)
car_autohome_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1881 entries, 0 to 1880
Data columns (total 29 columns):
slug                     1881 non-null object
name                     1881 non-null object
parent                   1881 non-null object
classified               1881 non-null object
mum                      1881 non-null object
car_autohome_model_id    1881 non-null int64
id                       1850 non-null float64
source_id                923 non-null float64
alias                    42 non-null object
url                      800 non-null object
checker_runtime_id       544 non-null float64
keywords                 1881 non-null object
classified_url           561 non-null object
slug_global              0 non-null float64
logo_img                 1827 non-null object
first_letter             1881 non-null object
has_detailmodel          1881 non-null int64
starting_price           1045 non-null float64
classified_slug          1549 non-null object
thumbnail                

In [90]:
origin_open_model_detail = pd.read_csv('./new_combine_detail.csv')
# origin_open_model_detail = origin_open_model_detail.loc[(origin_open_model_detail['status'] != 'D'),:]
origin_open_model_detail = origin_open_model_detail.drop(['detail_model_slug','detail_model','price_bn','global_slug','year','listed_year','volume','control','emission_standard','volume_extend'],axis=1)

car_autohome_detail = car_autohome_all.loc[:,['origin_model_detail_slug','detail_name','detail_slug','price_bn','origin_model_slug','online_year','listed_year','volume','control','emission_standard','volume_extend']].rename(columns={'origin_model_detail_slug':'detail_model_slug','detail_slug':'car_autohome_detail_id','detail_name':'detail_model','origin_model_slug':'global_slug','online_year':'year'})
car_autohome_detail = car_autohome_detail.merge(origin_open_model_detail,how='left',on=['car_autohome_detail_id'])
car_autohome_detail['created_on'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
car_autohome_detail['status'] = 'Y'
car_autohome_detail['has_param'] = 'Y'
car_autohome_detail.to_csv('./combine_detail.csv',index=False)
car_autohome_detail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32672 entries, 0 to 32671
Data columns (total 26 columns):
detail_model_slug         32672 non-null object
detail_model              32672 non-null object
car_autohome_detail_id    32672 non-null int64
price_bn                  32672 non-null float64
global_slug               32672 non-null object
year                      32672 non-null int64
listed_year               32672 non-null int64
volume                    32672 non-null float64
control                   32672 non-null object
emission_standard         32672 non-null object
volume_extend             32672 non-null object
id                        28599 non-null float64
source_id                 11586 non-null float64
checker_runtime_id        10606 non-null float64
old_dmodel                13536 non-null object
cont_vprice               17355 non-null float64
url                       10606 non-null object
domain                    20848 non-null object
status                  

In [3]:
test = pd.read_csv('./new_combine_detail.csv')
test = test.loc[:,['detail_model_slug']]
test = test.drop_duplicates(['detail_model_slug'])
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32672 entries, 0 to 32671
Data columns (total 1 columns):
detail_model_slug    32672 non-null object
dtypes: object(1)
memory usage: 510.5+ KB
